In [ ]:
BATCH_SIZE = 128

### Define Baseline Model

#### Define Input Function


This is the pipeline that will pull data from the training arrays (in memory) into the model graph

In [ ]:
def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    dataset = (dataset.shuffle(features.shape[0]) #Shuffle across the full number of examples
               .repeat(epochs = 1)
               .batch(batch_size))

    # Return the dataset.
    return dataset    
    
    

#### Define Graph


This is how we define the graph. In this case we pull out several metrics at evaulation time. 

In [2]:
def model_fn(features, labels, mode, params):
    
    input_layer = tf.reshape(features["x"], [-1, params['image_size'], params['image_size'], 3], name = "input_layer") #needs reshaping with new data
    
    #Convolutional Layer # 1
    conv1 = tf.layers.conv3d(
        inputs=input_layer,
        filters=32,
        kernel_size=params["kernel_size"],    #Kernal size too small for data?
        padding="same",
        activation=tf.nn.leaky_relu,
        name = "conv1")   
    
    #Average activations of convolutional layer 1
    with tf.variable_scope('Activations1'):
        average_density_1 = tf.reduce_mean(tf.reduce_sum(tf.cast((conv1 > 0), tf.float32), axis=[1]), name = "average_density_1")
        tf.summary.scalar('AvergageDensity1', average_density_1)
    
    #Pooling Layers #1
    pool1 = tf.layers.max_pooling3d(
        inputs=conv1, 
        pool_size=2,
        strides=2,
        name = "pool1")
    
    #Convolutional Layer # 2
    conv2 = tf.layers.conv3d(
        inputs=pool1,
        filters=64,
        kernel_size=params["kernel_size"],
        padding="same",
        activation=tf.nn.leaky_relu,
        name = "conv2")
    
    #Log the average activations of the second layer
    with tf.variable_scope('Activations1'):
        average_density_2 = tf.reduce_mean(tf.reduce_sum(tf.cast((conv2 > 0), tf.float32), axis=[1]), name = "average_density_2")
        tf.summary.scalar('AvergageDensity2', average_density_2)
    
    #Pooling layer # 2

    pool2 = tf.layers.max_pooling3d(
        inputs=conv2,
        pool_size=2,
        strides=2,
        name = "pool2")
    
    #Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, int(pool2.shape[2]) * int(pool2.shape[1] * int(pool2.shape(3)))])
    
    dense = tf.layers.dense(
        inputs=pool2_flat,
        units=128,
        activation=tf.nn.relu,
        name = "dense") #RELU NOT LEAKY!!!


        
    dropout = tf.layers.dropout(
        inputs=dense,
        rate=0.4,
        training = mode == tf.estimator.ModeKeys.TRAIN)
    
    
    #Logits layer
    
    logits = tf.layers.dense(
        inputs=dropout,
        units=2,
        name = "logits")
    
#--------------- MODEL OUTPUT STRUCTURES ----------------

    predictions = {

        #Generate Predictions (for PREDICT and EVAL mode)
        
        "classes": tf.argmax(input=logits, axis=1),
        
        #Add 'softmax_tensor' to the  graph. It is used for the
        #PREDICT by the 'logging_hook'
        
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    

    with tf.variable_scope('Accuracy'):
        labelsOH = tf.one_hot(labels, 2)
        correct_prediction = tf.equal(tf.argmax(tf.nn.softmax(logits), 1), tf.argmax(labelsOH, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
        #log the accuracy
        tf.summary.scalar('training_accuracy', accuracy)

    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    #Calculate Loss
    
    with tf.variable_scope('Loss_Layer'):	
        loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    #Create a logging hook for training metrics

    train_logging_hook = tf.train.SummarySaverHook(
        save_steps = 50,
        output_dir = params["dir"],
        summary_op = tf.summary.merge_all())



#     #Load up the image maps for conv1 from a checkpoint of the sparse encoder
#     tf.train.init_from_checkpoint(params["checkpoint"],
#                                   {'conv1/kernel':'conv1/kernel', #This overrides default initialization ops of specified variables.
#                                     'conv1/bias':'conv1/bias'})
    
    #Configure the training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        
        optimizer = tf.train.AdamOptimizer(learning_rate=params['learning_rate']) #CHANGED FOR SPARSITY MODEL
        train_op = optimizer.minimize(
            loss=loss, 
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(
            mode=mode, 
            loss=loss, 
            train_op=train_op, 
            training_hooks = [train_logging_hook]  #ADDED LOGGING HOOK
        ) 
    
    # Add evaluation metric (for EVAL mode), These give final performance metrics.
    
    eval_metric_ops = {
        "final_accuracy": tf.metrics.accuracy(
            labels=labels, predictions=predictions["classes"]), #Calculates how often the predictions matches the labels 
        
        "roc_auc_score": tf.metrics.auc( 
            labels = labels, predictions = predictions["classes"]), #Computes the approximate AUC via a Riemann sum
        
        "sensativity": tf.metrics.true_positives(
            labels = labels, predictions = predictions["classes"]), #Sum the weights of true-positives
        
        "false-positive (1 - specificity)": tf.metrics.false_positives(
            labels = labels, predictions = predictions["classes"]), #Sum the weights of false-positives
        
        "precision": tf.metrics.precision(
            labels = labels, predictions = predictions["classes"]) #Computes the precision of the predictions with respect to the labels.
    }
    

    return tf.estimator.EstimatorSpec(
        mode = mode,
        loss = loss,
        eval_metric_ops = eval_metric_ops)

Add params image_size and learning_rate